In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import TensorDataset, DataLoader

url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
df = pd.read_csv(url, header=None)

# Separate features and target
X = df.iloc[:, :-1].astype('float32')
y = df.iloc[:, -1]

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y).astype('float32')

In [31]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

In [32]:
# convert to torch tensors
X_train_tensor = torch.tensor(X_train.values)
y_train_tensor = torch.tensor(y_train).unsqueeze(1)  # Add dimension for compatibility
X_test_tensor = torch.tensor(X_test.values)
y_test_tensor = torch.tensor(y_test).unsqueeze(1)

# dataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# dataloader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)


In [19]:
import torch.nn as nn
import torch.nn.init as init

class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.hidden1 = nn.Linear(input_size, 10)
        init.kaiming_uniform_(self.hidden1.weight, nonlinearity='relu') # designed for ReLU
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(10, 8)
        init.kaiming_uniform_(self.hidden2.weight, nonlinearity='relu') # you must always remember to do this.
        self.act2 = nn.ReLU()
        self.hidden3 = nn.Linear(8, 1)
        init.xavier_uniform_(self.hidden3.weight) # designed for sigmoid, tanh activations. initialisations are important!
        self.act3 = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act3(self.hidden3(x))
        return x


In [20]:
import torch.optim as optim

def train_model(model, train_loader, epochs=100):
    criterion = nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    model.train()
    for epoch in range(epochs):
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()


In [33]:
from sklearn.metrics import accuracy_score
import numpy as np

def evaluate_model(model, test_loader):
    model.eval()
    predictions, actuals = [], []
    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            predicted = outputs.round()
            predictions.append(predicted.numpy()) # must be on cpu with nograd set.
            actuals.append(targets.numpy())
    predictions = np.vstack(predictions)
    actuals = np.vstack(actuals)
    accuracy = accuracy_score(actuals, predictions)
    return accuracy
X.shape

(351, 34)

In [22]:
model = MLP(input_size=X.shape[1]) # (samples, features) ; (row, cols) ; X = [X_i^T]

train_model(model, train_loader)

accuracy = evaluate_model(model, test_loader)
print(f'Accuracy: {accuracy:.3f}')

Accuracy: 0.853


# Details

1. Title: Johns Hopkins University Ionosphere database

2. Source Information:
   -- Donor: Vince Sigillito (vgs@aplcen.apl.jhu.edu)
   -- Date: 1989
   -- Source: Space Physics Group
              Applied Physics Laboratory
              Johns Hopkins University
              Johns Hopkins Road
              Laurel, MD 20723 

3. Past Usage:
   -- Sigillito, V. G., Wing, S. P., Hutton, L. V., \& Baker, K. B. (1989).
      Classification of radar returns from the ionosphere using neural 
      networks. Johns Hopkins APL Technical Digest, 10, 262-266.

      They investigated using backprop and the perceptron training algorithm
      on this database.  Using the first 200 instances for training, which
      were carefully split almost 50% positive and 50% negative, they found
      that a "linear" perceptron attained 90.7%, a "non-linear" perceptron
      attained 92%, and backprop an average of over 96% accuracy on the 
      remaining 150 test instances, consisting of 123 "good" and only 24 "bad"
      instances.  (There was a counting error or some mistake somewhere; there
      are a total of 351 rather than 350 instances in this domain.) Accuracy
      on "good" instances was much higher than for "bad" instances.  Backprop
      was tested with several different numbers of hidden units (in [0,15])
      and incremental results were also reported (corresponding to how well
      the different variants of backprop did after a periodic number of 
      epochs).

      David Aha (aha@ics.uci.edu) briefly investigated this database.
      He found that nearest neighbor attains an accuracy of 92.1%, that
      Ross Quinlan's C4 algorithm attains 94.0% (no windowing), and that
      IB3 (Aha \& Kibler, IJCAI-1989) attained 96.7% (parameter settings:
      70% and 80% for acceptance and dropping respectively).

4. Relevant Information:
   This radar data was collected by a system in Goose Bay, Labrador.  This
   system consists of a phased array of 16 high-frequency antennas with a
   total transmitted power on the order of 6.4 kilowatts.  See the paper
   for more details.  The targets were free electrons in the ionosphere.
   "Good" radar returns are those showing evidence of some type of structure 
   in the ionosphere.  "Bad" returns are those that do not; their signals pass
   through the ionosphere.  

   Received signals were processed using an autocorrelation function whose
   arguments are the time of a pulse and the pulse number.  There were 17
   pulse numbers for the Goose Bay system.  Instances in this databse are
   described by 2 attributes per pulse number, corresponding to the complex
   values returned by the function resulting from the complex electromagnetic
   signal.

5. Number of Instances: 351

6. Number of Attributes: 34 plus the class attribute
   -- All 34 predictor attributes are continuous

7. Attribute Information:     
   -- All 34 are continuous, as described above
   -- The 35th attribute is either "good" or "bad" according to the definition
      summarized above.  This is a binary classification task.

8. Missing Values: None